In [ ]:
import config
import sparknlp

spark = sparknlp.start()

In [ ]:
df = ( spark.read
    .format('mongodb')
    .option('database'      , config.MongoDb.database)
    .option('collection'    , config.MongoDb.collection)
    .option('connection.uri', f'mongodb://{config.MongoDb.user}:{config.MongoDb.password}@{config.MongoDb.host}:{config.MongoDb.port}')
    .load()
)
df.show(10)

In [ ]:
from sparknlp.pretrained import PretrainedPipeline 
pipeline = PretrainedPipeline.from_disk('./model')

In [ ]:
# result.select(
#     F.explode(
#       F.arrays_zip(
#         result.document.result, 
#         result.sentiment.result)).alias("cols")
# ).select(
#     F.expr("cols['0']").alias("document"),
#     F.expr("cols['1']").alias("sentiment")
# ).show(truncate=False
# )

In [ ]:
transformed_df = pipeline.transform(df)
transformed_df.show(10)

In [ ]:
( transformed_df.write
    .format('mongodb')
    .option('database'      , f'{config.MongoDb.database}')
    .option('collection'    , f'Sentiment')
    .option('connection.uri', f'mongodb://{config.MongoDb.user}:{config.MongoDb.password}@{config.MongoDb.host}:{config.MongoDb.port}')
    .mode('overwrite')
	.save()
)